# Ejemplo 2. Trabajando con imágenes

Construiremos un ejemplo de red neuronal muy simple, con apenas un par de capas, para abordar el famoso problema [MNIST](https://en.wikipedia.org/wiki/MNIST_database). Nuestro objetivo es ver el flujo de trabajo habitual para describir un modelo en Keras, pero sin profundizar en los detalles.

<img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png" width="300px" />

El primer paso es cargar la librería Keras. Podemos ver la versión que tenemos.

In [ ]:
import keras
print(keras.__version__)

## 1. Preparación de los datos

El primer paso de todo modelado es la preparación y carga de los datos. En nuestro caso, del problema [MNIST](http://yann.lecun.com/exdb/mnist/), que consta de una gran base de datos de dígitos escritos y que es tan habitual que se ha convertido en un ejemplo paradigmático dentro de Machine Learning.

El trabajo de preprocesamiento necesario para poder aplicar un modelo a este problema no es menor pero, afortunadamente, Keras proporciona una instrucción directa para descargar las imágenes que representan los miles de dígitos escritos a mano (ya con formato unificado de 28x28 pixels en escala de grises). Se pueden ver más datasets en su [web oficial](https://keras.io/api/datasets/)

Para poder cargar los datos que trae de ejemplo Keras hay que seguir dos pasos: primero, cargar la librería de Keras que proporciona las herramientas para trabajar con el dataset concreto (que suelen estar en el paquete `keras.datasets`, en este caso llamado `mnist`); y, segundo, ejecutar el proceso de carga de los datos (la librería proporciona la función `load_data()`). Ha de tenerse en cuenta que la primera vez que se realiza este proceso los datos se descargan desde un repositorio que viene por defecto predefinido en ese paquete, ya que, debido a su tamaño, no se instalan junto con la librería, sino únicamente cuando el usuario los necesita:

In [ ]:
from keras.datasets import mnist

(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

Observa que el proceso de carga de datos separa adecuadamente las diversas partes de que consta este dataset:  (_conjunto de entrenamiento_, _conjunto de test_), y cada uno de estos conjuntos está formado por un conjunto de datos (imágenes, en este caso concreto), con sus respectivas etiquetas de clasificación (_labels_). Además, aprovechamos la capacidad sintáctica de Python para realizar la carga de todos estos conjuntos en un solo paso (haciendo una 
asignación múltiple).

Podemos explorar un poco cómo son cada una de estas variables haciendo uso de instrucciones específicas de Python que nos dan información acerca de su estructura y muestra los primeros valores:

In [ ]:
train_data.shape

In [ ]:
len(train_labels)

In [ ]:
train_labels

In [ ]:
test_data.shape

In [ ]:
len(test_labels)

In [ ]:
test_labels

Si queremos ver alguna de las imágenes que hay en el dataset, podemos hacer uso de la siguiente función que hace uso de la librería `matplotlib`:

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

def gen_image(arr):
    conv = (np.reshape(arr, (28, 28)) * 255).astype(np.uint8)
    plt.imshow(conv, interpolation='nearest')
    plt.axis('off')
    return plt

# Dibujamos el primer ejemplo
gen_image(test_data[0]).show()

El flujo de trabajo es similar al que se sigue siempre en los procesos de ML Supervisado, y que hemos analizado en la introducción: 
  1. Mostramos al modelo (una red neuronal, en nuestro caso) los datos de *entrenamiento*, `train_data` y `train_labels`.
  2. El modelo debe *aprender* a asociar las imágenes con las etiquetas asociadas.
  3. Por último, verificamos el aprendizaje realizado comprobando sobre `test_data` que las respuestas dadas por el modelo (*predicciones*) coinciden con las almacenadas en `test_labels`.
  
  <img src="https://thumbs.gfycat.com/FickleHorribleBlackfootedferret-small.gif" />

## 2. Definición del modelo

Ya estamos en condiciones de definir una red neuronal que consumirá los datos anteriores para ver si somos capaces de dar una primera solución al problema del reconocimiento de dígitos manuscritos. Como solo estamos haciendo una primera aproximación a Keras, la red definida será muy básica, con solo una capa de entrada y una de salida:

  1. Vamos a situar una capa de entrada con 784 (= 28 * 28) neuronas (que recibirán cada uno de los 784 pixels de cada imagen), con función de activación ReLU, y 
  2. una capa de salida con 10 neuronas (una neurona para cada una de las posibles etiquetas de salida), y con activación softmax (por lo que se podrá interpretar como una probabilidad de salida que indica lo probable que es que la imagen de entrada tenga cada una de las etiquetas como salida).

In [ ]:
from keras import models
from keras import layers
from keras.utils.vis_utils import plot_model

red = models.Sequential()
red.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
red.add(layers.Dense(10, activation='softmax'))
plot_model(red, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

<!--<img src="https://raw.githubusercontent.com/miguelamda/DL/master/3.%20Frameworks%20Software/imgs/model_plot.png" />-->

Para facilitar la comprensión de esta introducción, hemos hecho uso de las utilidades de Keras para dar una representación visual de la estructura de la red. Ha de tenerse en cuenta que para ello es necesario instalar [Graphviz](http://www.graphviz.org) y la librería `pydot` de Python que se comunica con ella.

Además de las neuronas, que son las unidades atómicas que componen una red neuronal, desde un punto de vista funcional, el elemento básico de las redes neuronales es lo que se conoce como *capa* (*layer*), un módulo de procesamiento  formado por un conjunto de neuronas iguales que puede ver interpretarse como un "filtro" de datos. Como veremos a lo largo del curso, las capas son las encargadas de generar *representaciones* útiles de los datos que reciben, y que ayuden a resolver el problema para el que se ha construido la red. La mayor parte del Deep Learning, y donde esta demostrando un valor añadido respecto de los otros modelos de ML existentes, consiste en concatenar capas simples (y, posiblemente, con funcionalidades específicas diferenciadas) para obtener un dispositivo de cálculo que procesa datos de forma progresiva.

En el caso de la red que hemos definido, este dispositivo consta de una secuencia de dos capas densas, que son capas neurales totalmente conectadas. La segunda (y última) capa es una capa "softmax" de 10 salidas, lo que significa que devolverá un vector probabilístico de 10 valores (es decir, 10 valores en $[0,1]$ que suman 1). Cada uno de estos valores se interpretará la probabilidad de que la imagen actual pertenezca a una de las 10 clases (los dígitos del 0 al 9).

Hasta ahora solo hemos definido la estructura de la red, pero no hemos dado ninguna información acerca de cómo se llevará a cabo el entrenamiento. Para ello, hemos de indicarle a Keras algunas características adicionales, tales como el optimizador que permitirá modificar los pesos de la red, qué función objetivo (de error) se usará para dirigir esta optimización, y la métrica que usaremos para medir cómo se va comportando la red a medida que se entrena. 

Keras proporciona la función `compile` que permite establecer estas (y otras) propiedades sobre una red ya definida:


In [ ]:
red.compile(optimizer='Adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

En este caso estamos pasando todos los datos como cadenas que vienen predefinidas en Keras y trabajan con parámetros fijos, pero también es posible ajustar con más flexibilidad cada una de ellas y configurar los parámetros de los que depende, e incluso pasarle funciones, ya sean las que trae Keras o completamente personalizadas. Por ejemplo:

In [ ]:
from keras import optimizers
from keras import losses
from keras import metrics

red.compile(optimizer=optimizers.Adam(lr=0.001),
              loss=losses.categorical_crossentropy,
              metrics=[metrics.categorical_accuracy])


Como se puede observar, muchos de los chunks no proporcionan una salida imprimible, sino que crean nuevas variables o modifican el contenido de algunas de ellas para su reutilización posterior.

Debido a que la red neuronal que vamos a usar debe recibir como dato de entrada cada imagen de forma aplanada (es decir, no como una matriz de 28x28, sino como un vector de 28x28=784 posiciones), nuestro primer paso es hacer uso de las instrucciones que proporciona Keras para transformar la forma de los datos de entrada. Además, aprovecharemos para normalizar el contenido de estas imágenes (están en escalas de grises con valores `uint8` entre 0 y 255, y las pasaremos a valores `float32` en $[0,1]$), algo aconsejable cuando se trabaja con este tipo de modelos:

<img src="https://s3.amazonaws.com/media-p.slid.es/uploads/970798/images/5546082/ezgif.com-video-to-gif__1_.gif" />



In [ ]:
train_data = train_data.reshape((60000, 28 * 28))
train_data = train_data.astype('float32') / 255   # para imágenes es tan sencillo como dividir por 255, no es necesario usar normalize()

test_data = test_data.reshape((10000, 28 * 28))
test_data = test_data.astype('float32') / 255

Además, vamos a convertir las etiquetas (que vienen en el dataset como valores enteros), en vectores binarios con one-hot encoding para que se correspondan con la salida que nuestra red puede proporcionar. Si comparamos con el ejercicio 1, ahora vemos que es más sencillo porque los valores ya son enteros y no cadenas.

In [ ]:
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## 3. Proceso de entrenamiento

Preparados los datos y definida la red (estructura y funcionalidad), podemos hacer uso de la instrucción `fit` para comenzar el proceso de entrenamiento sobre los datos que tenemos. Esencialmente, hemos de indicar sobre qué datos entrenar (entrada y salidas), cuántas iteraciones (epochs) y con qué tamaño de batch (cada cuántos ejemplos el algoritmos actualiza los pesos).

Durante el proceso de entrenamiento, Keras informa de los valores que toma la función objetivo, así como de la/s métrica/s que hemos fijado en la compilación.

In [ ]:
red.fit(train_data, train_labels, epochs=5, batch_size=64)

Debemos tener en cuenta que los valores mostrados son el error y métricas calculados sobre los propios datos de entrenamiento. Sin embargo, como el objetivo de un modelo de aprendizaje es generalizar bien sobre datos que el proceso de entrenamiento no ha visto anteriormente, necesitamos el conjunto de test para evaluar cómo se comporta la red sobre ejemplos que no ha usado para ajustarse.

Sobre los datos de entrenamiento alcanzamos rápidamente una precisión de 0.989 (i.e. 98.9%), pero veamos cómo de bien se comporta con los datos de test (que no ha usado para aprender):

In [ ]:
test_loss, test_acc = red.evaluate(test_data, test_labels)

In [ ]:
print('test_acc:', test_acc)


Lo normal es que la red se comporte peor en los datos de test que en los datos de entrenamiento, ya que el proceso de entrenamiento consiste precisamente en ajustar los pesos para que el error cometido en estos últimos se minimice. Esta diferencia de comportamiento entre entrenamiento y test se denomina **overfitting** (o **sobreajuste**). En todo caso, con una red tan simple como la que hemos usado se alcanzan cotas de casi el 98% de aciertos.

Finalmente, podemos ver las predicciones que hace la red sobre algunos datos del conjunto de test (mostramos también las etiquetas aaociadas a los datos usados, pero ten en cuenta que están en formato binarizado, y el índice 1 corresponde a la etiqueta 0, el índice 2 a la etiqueta 1, etc...):

In [ ]:
def muestra_imagen_prediccion(i):
    # mostrar imagen
    gen_image(test_data[i]).show()
    # mostrar la etiqueta en modo categórico
    print("Etiqueta numérica:   ",test_labels[i:(i+1)])
    # mostrar la etiqueta en numérico
    print("Etiqueta categórica: ",np.argmax(test_labels[i:(i+1)]))
    # mostrar la predicción de la red
    print("Predicción:          ",np.argmax(red.predict(test_data[i:(i+1)])))


x = 70 # prueba aquí otro número para ver otro ejemplo
muestra_imagen_prediccion(x)



Puedes usar el siguiente código para ver k ejemplos mal predichos.

In [ ]:
k=4
for i in range(0,len(test_data)):
    if np.argmax(red.predict(test_data[i:(i+1)])) != np.argmax(test_labels[i:(i+1)]):
        k=k-1
        if k<0:
            break;
        muestra_imagen_prediccion(i)

## Ejercicio 1

Juega con el entrenamiento del modelo para ver qué resultados obtienes y si consigues mejorar (o por el contrario se empeora el resultado): cambia el método de [optimización](https://keras.io/api/optimizers/), amplía el número de epochs, añade capas, cambia [funciones de activación](https://keras.io/api/layers/), cambia el learning rate, etc. 

## Ejercicio 2

Vamos a probar con [CIFAR 10](https://keras.io/api/datasets/cifar10/), usando este mismo notebook y jugando con distintas redes, para ver hasta donde puedes llegar mejorando el resultado con lo que has aprendido hoy. Lleva cuidado que ahora las imágenes son a color y tienen 3 canales (RGB).

Primero, antes de comenzar, reinicia el entorno de ejecución. Esto limpiará las variables definidas. 

Segundo, comencemos cargando los datos:

In [ ]:
import keras
from keras.datasets import ?????

(train_data, train_labels), (test_data, test_labels) = ???????????????


Comprueba el número de ejemplos, y qué dimensiones tienen:

In [ ]:
print("dimensión del conjunto de entrenamiento: ", train_data.??????????)
print("dimensión del conjunto de test: ", test_data.?????????????)

# veamos algunas etiquetas
test_labels[:5]

Las etiquetas numéricas no nos dicen mucho, a continuación puedes usar este array para obtener el significado de cada una:

In [ ]:
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

Mostremos alguna imagen para hacernos una idea, pero tenemos que adaptar la función anterior a la forma de las imágenes de CIFAR10:


In [ ]:
from matplotlib import pyplot as plt
import numpy as np

def gen_image(arr):
    conv = (np.reshape(arr, (28, 28)) * 255).astype(np.uint8)  # adapta esta línea, es necesaria?
    plt.imshow(conv, interpolation='nearest')
    plt.axis('off')
    return plt

# Dibujamos el ejemplo i-ésimo del conjunto de test
i=10

gen_image(test_data[i]).show()
# Mostramos su etiqueta
print(label_names[test_labels[i][0]])

Define el modelo. Tenemos que aplanar las imágenes, que esta vez son de 32x32x3. Vamos a probar con una estructura parecida a la anterior:

In [ ]:
from keras import models
from keras import layers
from keras.utils.vis_utils import plot_model

red = models.??????????  #instancia un modelo secuencial
red.add(layers.???????(512, activation=????????, input_shape=??????????))   # añade una capa densa con función de activación ReLU, y adapta la capa de entrada
red.add(layers.?????(??????, activation=????????)) # añade una capa densa, ¿cuantos nodos necesitamos en la capa de salida?, ¿qué función de activación usaríamos?
plot_model(red, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

Ahora define el optimizador, la función de pérdida y la métrica a emplear, y compila el modelo:

In [ ]:
from keras import optimizers, losses, metrics

red.compile(optimizer=optimizers.???????(??????),      #usemos por ejemplo RMSprop y un learning rate del 0.0001
              loss=losses.categorical_crossentropy,    
              metrics=[metrics.categorical_accuracy])

Nos falta tratar los datos de entrada. Tenemos que aplanarlos y nomalizar sus valores:

In [ ]:
train_data = train_data.reshape((50000, ????????))
train_data = train_data.astype('float32') / 255   # para imágenes es tan sencillo como dividir por 255, no es necesario usar normalize()

test_data = test_data.reshape((10000, ??????????))
test_data = test_data.astype('float32') / 255

También tenemos que tratar las variables objetivo, o datos de salida. Tenemos que codificarlos con one-hot encoding:

In [ ]:
from keras.utils import ????????  # qué función necesitamos incluir para hacer one-hot encoding?

train_labels = ??????(train_labels)
test_labels = ???????(test_labels)

Ya estamos en disposición de entrenar el modelo. Lanza 10 épocas a ver qué tal:

In [ ]:
red.??????(??????, ????????, epochs=?????, batch_size=128)   # pasemos al entrenamiento

Finalmente, evaluémos lo obtenido sobre el conjunto de test:

In [ ]:
test_loss, test_acc = red.??????(test_data, test_labels)
print('test_acc:', test_acc)

Es posible que no salga un buen resultado... En tal caso, juega con los hiperparámetros, cambia el modelo, etc. Pero no llegarás muy lejos, hasta el próximo jueves...

Bien, vamos a ver algunos ejemplos:

In [ ]:
def muestra_imagen_prediccion(i):
    # mostrar imagen
    gen_image(test_data[i].reshape((32,32,3))).show()  #hay que volver a poner las imágenes como 32x32x3
    # mostrar la etiqueta 
    print("Etiqueta:       ",label_names[np.argmax(test_labels[i:(i+1)])])  
    # mostrar la predicción de la red
    print("Predicción:     ",label_names[np.argmax(red.predict(test_data[i:(i+1)]))])


x = 10 # prueba aquí otro número para ver otro ejemplo
muestra_imagen_prediccion(x)